# Import libraries

In [1]:
import os
import pandas as pd

# Read data

In [2]:
folder = os.path.join("..", "dataset-documentation", "raw_data")

In [3]:
data_18augsep = pd.read_csv(
    os.path.join(
        folder,
        "MBTA Bus Arrival Departure Times 2018",
        "MBTA Bus Arrival Departure Aug-Sept 2018.csv"
    )
)

In [4]:
data_18augsep.head()

,service_date,route_id,direction,half_trip_id,stop_id,time_point_id,time_point_order,point_type,standard_type,scheduled,actual,earliness,scheduled_headway,headway
0,2018-08-01T00:00:00Z,01,Inbound,40121394.0,75,mit,4.0,Midpoint,Schedule,1900-01-01T05:19:00Z,1900-01-01T05:19:34Z,-10.0,NaN,NaN
1,2018-08-01T00:00:00Z,01,Inbound,40121394.0,79,hynes,5.0,Midpoint,Schedule,1900-01-01T05:22:00Z,1900-01-01T05:23:20Z,3.0,NaN,NaN
2,2018-08-01T00:00:00Z,01,Inbound,40121394.0,187,masta,6.0,Midpoint,Schedule,1900-01-01T05:25:00Z,1900-01-01T05:25:58Z,-33.0,NaN,NaN
3,2018-08-01T00:00:00Z,01,Inbound,40121394.0,59,Wasma,7.0,Midpoint,Schedule,1900-01-01T05:28:00Z,1900-01-01T05:28:26Z,7.0,NaN,NaN
4,2018-08-01T00:00:00Z,01,Inbound,40121565.0,110,hhgat,1.0,Startpoint,Headway,1900-01-01T05:30:00Z,1900-01-01T05:29:57Z,NaN,1200.0,1218.0


# Data cleaning

## drop "earliness" variable

In [5]:
if "earliness" in data_18augsep.columns:
    data_18augsep = data_18augsep.drop("earliness", axis = 1)

## reformat "service_date" variable

In [6]:
data_18augsep["service_date"] = pd.to_datetime(data_18augsep["service_date"])
# keep Year-Month-Day
data_18augsep['service_date'] = data_18augsep['service_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [7]:
## reformat scheduled & actual variables
## also add a nextday boolean to indicate whether the scheduled and actual is on the next day
data_18augsep['scheduled'] = pd.to_datetime(data_18augsep['scheduled'])
data_18augsep['scheduled_nextday'] = data_18augsep['scheduled'].apply(lambda x: (x.strftime('%m-%d') if pd.notnull(x) else x) == '01-02')
data_18augsep['scheduled'] = data_18augsep['scheduled'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notnull(x) else x)

In [8]:
data_18augsep['actual'] = pd.to_datetime(data_18augsep['actual'])
data_18augsep['actual_nextday'] = data_18augsep['actual'].apply(lambda x: (x.strftime('%m-%d') if pd.notnull(x) else x ) == '01-02')
data_18augsep['actual'] = data_18augsep['actual'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notnull(x) else x)

In [9]:
data_18augsep.head()

,service_date,route_id,direction,half_trip_id,stop_id,time_point_id,time_point_order,point_type,standard_type,scheduled,actual,scheduled_headway,headway,scheduled_nextday,actual_nextday
0,2018-08-01,01,Inbound,40121394.0,75,mit,4.0,Midpoint,Schedule,05:19:00,05:19:34,NaN,NaN,False,False
1,2018-08-01,01,Inbound,40121394.0,79,hynes,5.0,Midpoint,Schedule,05:22:00,05:23:20,NaN,NaN,False,False
2,2018-08-01,01,Inbound,40121394.0,187,masta,6.0,Midpoint,Schedule,05:25:00,05:25:58,NaN,NaN,False,False
3,2018-08-01,01,Inbound,40121394.0,59,Wasma,7.0,Midpoint,Schedule,05:28:00,05:28:26,NaN,NaN,False,False
4,2018-08-01,01,Inbound,40121565.0,110,hhgat,1.0,Startpoint,Headway,05:30:00,05:29:57,1200.0,1218.0,False,False


# Missing values

In [10]:
data_18augsep.isna().sum() / data_18augsep.shape[0]

service_date         0.000000
route_id             0.000000
direction            0.000000
half_trip_id         0.000030
stop_id              0.000000
time_point_id        0.000234
time_point_order     0.000234
point_type           0.000000
standard_type        0.000000
scheduled            0.000000
actual               0.070866
scheduled_headway    0.506128
headway              0.581525
scheduled_nextday    0.000000
actual_nextday       0.000000
dtype: float64

In [11]:
## "half_trip_id" missing values (0.0030%)
data_18augsep = data_18augsep.dropna(subset=["half_trip_id"])

In [12]:
## "time_point_id" & "time_point_order" missing values (0.0234%)
## We can use others years data to interpolate, but the time_point_order might be different. 
## So we just drop the missing vaue
data_18augsep = data_18augsep.dropna(subset=["time_point_id", "time_point_order"])

In [13]:
## "actual" missing values (7.0866%)
data_18augsep = data_18augsep.dropna(subset=["actual"])

In [14]:
## based on data desc "scheduled_headway is NA when the "standard_type" is Headway, so we do not need to do anything to this var
data_18augsep.query("standard_type == 'Headway'")["scheduled_headway"].isna().sum()

np.int64(0)

In [15]:
## "headway" should not have missing value if the "standard_type" is "Headway", however, we found there are many reason will cause the headway missing.
## so we just keep the missing values first
num = data_18augsep.query("standard_type == 'Headway'")["headway"].isna().sum()
print(f'Num of Headway missing values, when standard_type is Headway: {num}')

Num of Headway missing values, when standard_type is Headway: 166320


In [16]:
data_18augsep = data_18augsep.reset_index(drop=True)
data_18augsep

,service_date,route_id,direction,half_trip_id,stop_id,time_point_id,time_point_order,point_type,standard_type,scheduled,actual,scheduled_headway,headway,scheduled_nextday,actual_nextday
0,2018-08-01,01,Inbound,40121394.0,75,mit,4.0,Midpoint,Schedule,05:19:00,05:19:34,NaN,NaN,False,False
1,2018-08-01,01,Inbound,40121394.0,79,hynes,5.0,Midpoint,Schedule,05:22:00,05:23:20,NaN,NaN,False,False
2,2018-08-01,01,Inbound,40121394.0,187,masta,6.0,Midpoint,Schedule,05:25:00,05:25:58,NaN,NaN,False,False
3,2018-08-01,01,Inbound,40121394.0,59,Wasma,7.0,Midpoint,Schedule,05:28:00,05:28:26,NaN,NaN,False,False
4,2018-08-01,01,Inbound,40121565.0,110,hhgat,1.0,Startpoint,Headway,05:30:00,05:29:57,1200.0,1218.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4753795,2018-09-30,SL5,Outbound,40832424.0,15176,mawor,8.0,Midpoint,Headway,00:54:00,01:27:08,900.0,2609.0,True,True
4753796,2018-09-30,SL5,Outbound,40832424.0,55,Wasma,9.0,Midpoint,Headway,00:55:00,01:27:46,900.0,2611.0,True,True
4753797,2018-09-30,SL5,Outbound,40832424.0,60,Walen,10.0,Midpoint,Headway,00:56:00,01:28:07,900.0,2610.0,True,True
4753798,2018-09-30,SL5,Outbound,40832424.0,61,Melwa,11.0,Midpoint,Headway,00:57:00,01:29:13,900.0,2615.0,True,True


In [17]:
data_18augsep.to_csv("arrdep_18augsep_simplecleaned.csv", index=False)

In [18]:
# data cleaning pipeline
def clean_data(df):
    if 'earliness' in df.columns:
        df = df.drop('earliness', axis = 1)
    df['service_date'] = pd.to_datetime(df['service_date'])
    df['service_date'] = df['service_date'].apply(lambda x: x.strftime('%Y-%m-%d')) 

    df['scheduled'] = pd.to_datetime(df['scheduled'])
    df['scheduled_nextday'] = df['scheduled'].apply(lambda x: x.strftime('%m-%d') == '01-02')
    df['scheduled'] = df['scheduled'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notnull(x) else x)
    df['actual'] = pd.to_datetime(df['actual'])
    df['actual_nextday'] = df['actual'].apply(lambda x: x.strftime('%m-%d') == '01-02')
    df['actual'] = df['actual'].apply(lambda x: x.strftime('%H:%M:%S') if pd.notnull(x) else x)

    df = df.dropna(subset=['half_trip_id', 'time_point_id', 'time_point_order', 'actual'])
    return df